<a href="https://colab.research.google.com/github/emtruong2/csc422-spring2024/blob/main/DT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading Libraries and Data

In [ ]:
import numpy as np
import pandas as pd
from typing import List

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get the directory path
dir = ('/content/drive/My Drive/Junior/Spring 2024/CSC 422/CSC 422 Project/Datasets/')

# Get the data
originalFraudTrain = pd.read_csv(dir + 'fraudTrain.csv')
originalFraudTest = pd.read_csv(dir + 'fraudTest.csv')

In [ ]:
originalFraudTrain.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Preprocessing Data

In [ ]:
# Set a seed for reproducibility
random_seed = 24
np.random.seed(random_seed)

In [ ]:
columns = originalFraudTrain.columns
columns

NameError: name 'originalFraudTrain' is not defined

In [ ]:
def dropColumns(df: pd.DataFrame) -> pd.DataFrame:
  """
  Remove unnecessary and personally identifiable information (PII) columns from
  the DataFrame.

  :param df: the DataFrame
  :return: the DataFrame without unnecessary and PII columns
  """

  df = df.drop(columns=['Unnamed: 0', 'trans_date_trans_time', 'first',  'last',
                        'gender', 'street', 'city', 'state', 'zip', 'lat', 'long',
                        'city_pop', 'job', 'dob', 'trans_num'])

  return df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalize(df: pd.DataFrame) -> pd.DataFrame:
  """
  Use Min-Max scaling to scale numeric features to 0-1.
  Formula: (X - Xmin) / (Xmax - Xmin)

  :param df: the DataFrame
  :return: the DataFrame with numeric features scaled
  """
  columns = ['amt', 'unix_time','merch_lat', 'merch_long']
  df[columns] = MinMaxScaler().fit_transform(df[columns])

  return df

In [ ]:
# Drop unnecessary and PII columns
fraudTrain = dropColumns(originalFraudTrain)
fraudTest = dropColumns(originalFraudTest)

# Scale numeric features to 0-1
fraudTrain = normalize(fraudTrain)
fraudTest = normalize(fraudTest)

In [ ]:
fraudTrain.head()

,cc_num,merchant,category,amt,unix_time,merch_lat,merch_long,is_fraud
0,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,0.000137,0.000000e+00,0.350302,0.848602,0
1,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,0.003670,5.598526e-07,0.621488,0.486208,0
2,38859492057661,fraud_Lind-Buckridge,entertainment,0.007569,7.105821e-07,0.497559,0.546697,0
3,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,0.001520,1.248902e-06,0.577663,0.542619,0
4,375534208663984,fraud_Keeling-Crist,misc_pos,0.001415,3.617509e-06,0.405244,0.882857,0


## Processing Data

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def train_decision_tree(X_train, Y_train, X_test, Y_test):
    dt_classifier = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=random_seed)

    # Train the model
    dt_classifier.fit(X_train, Y_train)

    # Make predictions
    predictions = dt_classifier.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(Y_test, predictions)
    precision = precision_score(Y_test, predictions)
    recall = recall_score(Y_test, predictions)
    f1 = f1_score(Y_test, predictions)

    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)

    return dt_classifier

# Data stored in fraudTrain and fraudTest
X_train = fraudTrain.drop(columns=['is_fraud'])
Y_train = fraudTrain['is_fraud']
X_test = fraudTest.drop(columns=['is_fraud'])
Y_test = fraudTest['is_fraud']

# Train and evaluate the decision tree
dt_model = train_decision_tree(X_train, Y_train, X_test, Y_test)


ValueError: could not convert string to float: 'fraud_Rippin, Kub and Mann'

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

def plot_tree(model):
    plt.figure(figsize=(20,10))
    tree.plot_tree(model, filled=True, feature_names=X_train.columns, class_names=['Not Fraud', 'Fraud'], rounded=True, fontsize=12)
    plt.show()

plot_tree(dt_model)


NameError: name 'dt_model' is not defined